https://blog.csdn.net/qq_47273708/article/details/134228655?spm=1001.2101.3001.6650.4&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7ERate-4-134228655-blog-124489562.235%5Ev43%5Epc_blog_bottom_relevance_base6&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7ERate-4-134228655-blog-124489562.235%5Ev43%5Epc_blog_bottom_relevance_base6&utm_relevant_index=9

数据预处理

In [1]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

 
 
 
# S: 起始标记
# E: 结束标记
# P：意为padding，将当前序列补齐至最长序列长度的占位符
sentence = [
    # enc_input   dec_input    dec_output
    ['ich mochte ein bier P','S i want a beer .', 'i want a beer . E'],
    ['ich mochte ein cola P','S i want a coke .', 'i want a coke . E'],
]
 
# 词典，padding用0来表示
# 源词典
src_vocab = {'P':0, 'ich':1,'mochte':2,'ein':3,'bier':4,'cola':5}
src_vocab_size = len(src_vocab) # 6
# 目标词典（包含特殊符）
tgt_vocab = {'P':0,'i':1,'want':2,'a':3,'beer':4,'coke':5,'S':6,'E':7,'.':8}
# 反向映射词典，idx ——> word (序号和对应的词)
idx2word = {v:k for k,v in tgt_vocab.items()}
tgt_vocab_size = len(tgt_vocab) # 9
 
src_len = 5 # 输入序列enc_input的最长序列长度，其实就是最长的那句话的token数
tgt_len = 6 # 输出序列dec_input/dec_output的最长序列长度

# 构建模型输入的Tensor
def make_data(sentence):
    enc_inputs, dec_inputs, dec_outputs = [],[],[]
    for i in range(len(sentence)):
        enc_input = [src_vocab[word] for word in sentence[i][0].split()]
        dec_input = [tgt_vocab[word] for word in sentence[i][1].split()]
        dec_output = [tgt_vocab[word] for word in sentence[i][2].split()]
        
        enc_inputs.append(enc_input)
        dec_inputs.append(dec_input)
        dec_outputs.append(dec_output)
        
    # LongTensor是专用于存储整型的，Tensor则可以存浮点、整数、bool等多种类型
    return torch.LongTensor(enc_inputs),torch.LongTensor(dec_inputs),torch.LongTensor(dec_outputs)
 
enc_inputs, dec_inputs, dec_outputs = make_data(sentence)
 
print(' enc_inputs: \n', enc_inputs)  # enc_inputs: [2,5]
print(' dec_inputs: \n', dec_inputs)  # dec_inputs: [2,6]
print(' dec_outputs: \n', dec_outputs) # dec_outputs: [2,6]

 enc_inputs: 
 tensor([[1, 2, 3, 4, 0],
        [1, 2, 3, 5, 0]])
 dec_inputs: 
 tensor([[6, 1, 2, 3, 4, 8],
        [6, 1, 2, 3, 5, 8]])
 dec_outputs: 
 tensor([[1, 2, 3, 4, 8, 7],
        [1, 2, 3, 5, 8, 7]])


加载数据

In [2]:
# 使用Dataset加载数据
class MyDataSet(Data.Dataset):
    def __init__(self,enc_inputs, dec_inputs, dec_outputs):
        super(MyDataSet,self).__init__()
        self.enc_inputs = enc_inputs
        self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs
        
    def __len__(self):
        # 我们前面的enc_inputs.shape = [2,5],所以这个返回的是2
        print(enc_inputs.shape[0]) #矩阵shape就是 0：行 1：列
        return self.enc_inputs.shape[0] 
    
    # 根据idx返回的是一组 enc_input, dec_input, dec_output
    def __getitem__(self, idx):
        return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]
 
# 构建DataLoader
loader = Data.DataLoader(dataset=MyDataSet(enc_inputs,dec_inputs, dec_outputs),batch_size=2,shuffle=True)

2
2


模型参数
Transformer包含Encoder和Decoder

Encoder和Decoder各自包含6个Layer

Encoder Layer中包含 Self Attention 和 FFN 两个Sub Layer

Decoder Layer中包含 Masked Self Attention、 Cross Attention、 FFN 三个Sub Layer


In [3]:
# 用来表示一个词的向量长度
d_model = 512
 
# FFN的隐藏层神经元个数
d_ff = 2048
 
# 分头后的q、k、v词向量长度，依照原文我们都设为64
# 原文：queries and kes of dimention d_k,and values of dimension d_v .所以q和k的长度都用d_k来表示
d_k = d_v = 64
 
# Encoder Layer 和 Decoder Layer的个数
n_layers = 6
 
# 多头注意力中head的个数，原文：we employ h = 8 parallel attention layers, or heads
n_heads = 8

Positional Encoding
用于为输入的词向量进行位置编码
The positional encodings have the same dimension d_model as the embeddings, so that the two can be summed


In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000): 
        # dropout是原文的0.1，max_len原文没找到
        #max_len是假设的一个句子最多包含5000个token
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # 开始位置编码部分,先生成一个max_len * d_model 的矩阵，即5000 * 512
        # 5000是一个句子中最多的token数，512是一个token用多长的向量来表示，5000*512这个矩阵用于表示一个句子的信息
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # pos：[max_len,1],即[5000,1]
        # 先把括号内的分式求出来,pos是[5000,1],分母是[256],通过广播机制相乘后是[5000,256]
        div_term = pos / pow(10000.0,torch.arange(0, d_model, 2).float() / d_model)
        
        # 再取正余弦
        pe[:, 0::2] = torch.sin(div_term)
        pe[:, 1::2] = torch.cos(div_term)
        # 一个句子要做一次pe，一个batch中会有多个句子，所以增加一维用来和输入的一个batch的数据相加时做广播
        pe = pe.unsqueeze(0) # [5000,512] -> [1,5000,512] 
        # 将pe作为固定参数保存到缓冲区，不会被更新
        self.register_buffer('pe', pe)
        
        
    def forward(self, x):
        '''x: [batch_size, seq_len, d_model]'''
        # 5000是我们预定义的最大的seq_len，就是说我们把最多的情况pe都算好了，用的时候用多少就取多少
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x) # return: [batch_size, seq_len, d_model], 和输入的形状相同


Mask  防止关注padding

In [5]:
# 为enc_input和dec_input做一个mask，把占位符P的token（就是0） mask掉
# 返回一个[batch_size, len_q, len_k]大小的布尔张量，True是需要mask掉的位置
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size() #batchsize 和lenq 等于seqq的长宽
    batch_size, len_k = seq_k.size()
    # seq_k.data.eq(0)返回一个等大的布尔张量，seq_k元素等于0的位置为True,否则为False
    # 然后扩维以保证后续操作的兼容(广播)
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1) # pad_attn_mask: [batch_size,1,len_k]
    # 要为每一个q提供一份k，所以把第二维度扩展了q次
    # 另注意expand并非真正加倍了内存，只是重复了引用，对任意引用的修改都会修改原始值
    # 这里是因为我们不会修改这个mask所以用它来节省内存
    return pad_attn_mask.expand(batch_size, len_q, len_k) # return: [batch_size, len_q, len_k]
    # 返回的是batch_size个 len_q * len_k的矩阵，内容是True和False，
    # 第i行第j列表示的是query的第i个词对key的第j个词的注意力是否无意义，若无意义则为True，有意义的为False（即被padding的位置是True）

**Sequence mask**

In [6]:
# 用于获取对后续位置的掩码，防止在预测过程中看到未来时刻的输入
# 原文：to prevent positions from attending to subsequent positions
def get_attn_subsequence_mask(seq):
    """seq: [batch_size, tgt_len]"""
    # batch_size个 tgt_len * tgt_len的mask矩阵
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    # np.triu 是生成一个 upper triangular matrix 上三角矩阵，k是相对于主对角线的偏移量
    # k=1意为不包含主对角线（从主对角线向上偏移1开始）
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)
    subsequence_mask = torch.from_numpy(subsequence_mask).byte() # 因为只有0、1所以用byte节省内存
    return subsequence_mask  # return: [batch_size, tgt_len, tgt_len]

scaled dot-product attention此函数用于计算缩放点积注意力，在MultiHeadAttention中被调用

In [7]:
class ScaledDotProductionAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductionAttention,self).__init__()

    def forward(self, Q, K, V, attn_mask):
        ''' 
         Q [batch size, n_heads, len_q, d_k]
         K [batch size, n_heads, len_k, d_k]
         V [batch size, n_heads, len_v=len_k, d_k]
         全文两处用到注意力，一处是self attention，
         另一处是co attention，
         前者不必说，后者的k和v都是encoder的输出，所以k和v的形状总是相同的
         attn_mask: [batch size, n_heads, seq_len, seq_len]
        '''
         #1) 计算attention score = QK^T/根号dk
         #K.shape = [batch_size, n_heads, len_k, d_k]
         #K^T:
         #K.shape → [batch_size, n_heads, d_k, len_k]

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        
        # scores: [batch_size, n_heads, len_q, len_k] transpose(-1,-2)是交换后面两个维度

        #2) 进行mask 和 softmax
        scores.masked_fill_(attn_mask, -1e9)# mask为True的位置会被设为-1e9
        attn = nn. Softmax(dim=-1)(scores) # attn: [batch_size, n_heads, len_q, len_k]
        # 3) 乘V得到最终的加权
        context = torch.matmul(attn, V) # context: [batch_size, n_heads, len_q, d_v]
        '''
        得出的context是每个维度(d_1-d_v)都考虑了在当前维度(这一列)当前token对所有token的注意力后更新的新的值，
        换言之每个维度d是相互独立的，每个维度考虑自己的所有token的注意力，所以可以理解成1列扩展到多列
        返回的context: [batch_size, n_heads, len_q, d_v]本质上还是batch_size个句子，
        只不过每个句子中词向量维度512被分成了8个部分，分别由8个头各自看一部分，
        每个头算的是整个句子(一列)的512/8=64个维度，最后按列拼接起来
        '''
        return context



Multihead Attention

In [8]:
class MultiHeadAttention(nn.Module):
    def _init_(self):
        super(MultiHeadAttention,self)._init_()
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        self.concat = nn.Linear(d_model, d_model)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        ''' 
        input_Q:[batch_size, len_q, d_model]
        len_q是作为query的句子的长度，比如enc_inputs（2,5,512）作为输入，那句子长度5就是len_q
        input_K:[batch_size, len_k, d_model]
        input_V:[batch_size, len_v(len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        residual, batch_size = input_Q, input_Q.size(0)
        # 1）linear projection 
        # 用 W_Q 线性变换 input_Q，把 d_model 维度映射到 n_heads * d_k 维度。
        # 用 .view() 重新 reshape 形状，把 n_heads 维度拆分出来，使其可以进行多头计算。
        # input_Q [batch_size, seq_len, d_model] ->  [batch_size, n_heads, seq_len, d_k/d_v]

        # .view(batch_size, -1, n_heads, d_k) 重新 reshape：
        # -1 表示保持 seq_len 维度不变
        # n_heads, d_k 把注意力头数 n_heads 分离出来
        # 变换后形状：[batch_size, seq_len, n_heads, d_k]
        #然后再进行转1，2 交换 nheads和seqlen

        

        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1, 2) 
    
        # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1, 2) 
        # V: [batch_size, n_heads, len_v(=len_k), d_v]

        #2）计算attention
        # 自我复制n_heads次，为每个头准备一份mask
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        # attn_mask: [batch_size, n_heads, seq_len, seq_len]
        context = ScaledDotProductionAttention()(Q, K, V, attn_mask)
        # context: [batch_size, n_heads, len_q, d_v]

        #3)concat
        context = torch.cat([context[:,i,:,:] for i in range (context.size(1))], dim=-1)
        output = self.concat(context)# [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).cuda()(output + residual)
    #output: [batch_size, len_q, d_model]
    #context = context.transpose(1.2).reshape(batch_size, -1, d_model)
    #output = self.linear(context)

 






FeedForward Networks


In [9]:
class PositionwiseFeedForward(nn.Module):
    def _init_(self):
        super(PositionalEncoding,self).__init__()
        #MLP
        self.fc = nn.Sequential(
            nn.Linear(d_model,d_ff),
            nn.ReLU(),
            nn.Linear(d_ff,d_model)

        )
    def forward(self,inputs):
        #inputs:[b_s, seq_len, d_m]
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).cuda()(output + residual )
    # return： [batch_size, seq_len, d_model] 形状不变



    '''
    class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x  # 残差连接
        x = self.fc1(x)  # 第一层线性变换
        x = self.relu(x)  # 非线性激活
        x = self.fc2(x)  # 第二层线性变换
        return self.layer_norm(x + residual)  # 残差连接 + 归一化
'''

encoder layer

In [10]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer,self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PositionwiseFeedForward()

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs:[bs, erc_len, d_model]
        enc_self_attn_mask: [bs, src_len, src_len]
        QKV 均为 enc_inputs

        '''

        enc_outputs = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        enc_outputs = self.pos_ffn(enc_outputs) 
        # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs # enc_outputs: [batch_size, src_len, d_model]

Encoder Stack封装

In [11]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        # 直接调的现成接口完成词向量的编码，输入是类别数和每一个类别要映射成的向量长度
        self.src_emb = nn.Embedding(src_vocab_size, d_model)
        #定义 输入词嵌入层（Word Embedding）。
        # src_vocab_size：源语言（德语）词汇表大小。

        self.pos_emb = PositionalEncoding(d_model)
        #位置编码层，给输入 Token 添加位置信息。

        self.layer = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])#重复运行layers数((六次))
        #创建 n_layers=6 层 Encoder。
        #nn.ModuleList([]) 用于存储多个 EncoderLayer() 层。

        '''等价于
            self.layers = nn.ModuleList([
                EncoderLayer(),
                EncoderLayer(),
                EncoderLayer(),
                EncoderLayer(),
                EncoderLayer(),
                EncoderLayer()
                ])'''
    def forward(self, enc_inputs):
        ''' enc_inputs: [batchsize, src_len]'''
        enc_outputs = self.src_emb(enc_inputs)
        # [batch_size, src_len] -> [batch_size, src_len, d_model]
        # 把 enc_inputs（Token ID）转换成 d_model 维向量。
        enc_outputs = self.pos_emb(enc_outputs)
        #加上位置编码，确保模型能识别 Token 的顺序。
        # enc_outputs: [batch_size, src_len, d_model]

        # Encoder中是self attention, 传入QK都是enc_inputs
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs,enc_inputs)
        # enc_self_attn_mask: [batch_size, src_len, src_len]计算mask
        for layer in self.layers:
            enc_outputs = layers(enc_outputs,enc_self_attn_mask)
        return enc_outputs
    # enc_outputs: [batch_size, src_len, d_model]
    



Decoder layer


In [12]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super (DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PositionwiseFeedForward()
    def forward(self,dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):

        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        dec_enc_attn_mask: [batch_size, tgt_len, src_len] 前者是Q后者是K

        '''


        dec_outputs = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs)

        return dec_outputs
    # dec_outputs: [batch_size, tgt_len, d_model]

decoder

In [13]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size,d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range (n_layers)])

    def forward (self, dec_inputs, enc_inputs,enc_outputs):

        '''
        这三个参数对应的不是Q、K、V，dec_inputs是Q，enc_outputs是K和V，enc_inputs是用来计算padding mask的
        dec_inputs: [batch_size, tgt_len]
        enc_inpus: [batch_size, src_len]
        enc_outputs: [batch_size, src_len, d_model]

        '''
        #forward() 计算 Decoder 输出
        dec_outputs = self.tgt_emb(dec_inputs)
        dec_outputs = self.pos_emb(dec_outputs)


        #计算 Decoder Self-Attention 的 Mask
        
        #屏蔽pad位置
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs,dec_inputs).cuda()

        #屏蔽未来信息
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).cuda()

        # 将两个mask叠加，布尔值可以视为0和1，和大于0的位置是需要被mask掉的，赋为True，
        # 和为0的位置是有意义的为False
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask),0).cuda()


        #计算Enc-Decattention的Mask

        #co-attention 部分 传入的是enc_inputs 
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)
        #enc_inputs 计算pad mask 
        #enc_output 计算QK相关性

        '''
        get_attn_pad_mask(seq_q, seq_k) 计算的是 K 的 PAD Mask，用于屏蔽 PAD 位置的注意力分数。
        在 Encoder-Decoder Attention 里：
        Q（Query） 来自 Decoder（dec_inputs）。
        K 和 V（Key & Value） 来自 Encoder（enc_outputs）。
        enc_outputs 和 enc_inputs 形状相同，enc_outputs 是 enc_inputs 经过 Encoder 处理后的表示，
        但 PAD 位置没变。
        Mask 只关心哪些位置是 PAD，所以 enc_inputs 可以直接用来计算 PAD 位置，而不需要 enc_outputs。
        '''

        




        for layer in self.layers:
            dec_inputs = layer(dec_inputs,enc_inputs, dec_self_attn_mask, dec_enc_attn_mask)
            
        return dec_outputs  # [batch_size, tgt_len, d_model]







整合整个transformer

In [14]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder().cuda()
        self.decoder = Decoder().cuda()
        self.projection = nn.Linear(d_model, tgt_vocab_size).cuda()

    def forward(self,enc_inputs,dec_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        '''
        enc_outputs = self.encoder(enc_inputs)
        dec_outputs = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        dec_logits = self.projection(dec_outputs)

        # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        # 解散batch，一个batch中有batch_size个句子，每个句子有tgt_len个词（即tgt_len行），
        # 现在让他们按行依次排布，如前tgt_len行是第一个句子的每个词的预测概率，
        # 再往下tgt_len行是第二个句子的，一直到batch_size * tgt_len行

        return dec_logits.view(-1,dec_logits.size(-1))#  [batch_size * tgt_len, tgt_vocab_size]
        '''最后变形的原因是：nn.CrossEntropyLoss接收的输入的第二个维度必须是类别
        dec_logits.size(-1) 代表 tgt_vocab_size，即 dec_logits 的最后一维。
        -1 让 PyTorch 自动计算 前两维 batch_size * tgt_len 的大小。
        最终形状变为 [batch_size * tgt_len, tgt_vocab_size]。
        
        '''




训练过程


In [15]:
model = Transformer().cuda()
model.train()

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(),lr = 1e-3, momentum=0.99)


for epoch in range(1000):
    for enc_inputs, dec_inputs, dec_outputs in loader:
        '''
        enc_inputs: [batch_size, src_len] [2,5]
        dec_inputs: [batch_size, tgt_len] [2,6]
        dec_outputs: [batch_size, tgt_len] [2,6]
        '''
        enc_inputs, dec_inputs, dec_outputs = enc_inputs.cuda(), dec_inputs.cuda()
        outputs = model (enc_inputs, dec_inputs)
        #outputs: [batch_size * tgt_len, tgt_vocab_size], dec_outputs: [batch_size, tgt_len]
        loss = criterion(outputs,dec_outputs.view(-1))# 将dec_outputs展平成一维张量
        #更新权重
        optimizer.zero_gard()
        loss.backward()
        optimizer.step()
        print(f'Epoch[{epoch+1}/1000], Loss:{loss.item()}')
torch.save(model,'MyPt.pth')








AssertionError: Torch not compiled with CUDA enabled

In [16]:
import torch
print(torch.cuda.is_available())  # 检查是否支持 CUDA
print(torch.__version__)          # 检查 PyTorch 版本
print(torch.version.cuda)         # 检查 CUDA 版本
print(torch.cuda.device_count())

False
2.5.1+cpu
None
0
